In [1]:
import requests
import pandas as pd

In [2]:
response = requests.get('https://www.forbes.com/forbesapi/org/top-colleges/2021/position/true.json?limit=1000&fields=organizationName,academics,state,financialAid,rank,medianBaseSalary,campusSetting,studentPopulation,organization,description')

responseObject = response.json()

responseData = responseObject['organizationList']['organizationsLists']

data_main = pd.DataFrame(responseData)

In [3]:
data_main

,description,rank,organization,organizationName,state,studentPopulation,academics,financialAid,campusSetting,medianBaseSalary
0,One of the top public universities in the coun...,1,"{'naturalId': 'fred/college/64', 'name': 'Univ...","University of California, Berkeley",CA,43185,"{'type': 'Public', 'carnegieClassification': '...","{'totalGrantAid': 61548859.0, 'percentOfStuden...",Urban,138800.0
1,"The second oldest Ivy League institution, Yale...",2,"{'naturalId': 'fred/college/10', 'name': 'Yale...",Yale University,CT,13609,"{'type': 'Private', 'carnegieClassification': ...","{'totalGrantAid': 48792777.0, 'percentOfStuden...",Urban,141300.0
2,Princeton is a leading private research univer...,3,"{'naturalId': 'fred/college/2', 'name': 'Princ...",Princeton University,NJ,8419,"{'type': 'Private', 'carnegieClassification': ...","{'totalGrantAid': 43368213.0, 'percentOfStuden...",Suburban,150500.0
3,"Located just outside of Palo Alto, CA, Stanfor...",4,"{'naturalId': 'fred/college/6', 'name': 'Stanf...",Stanford University,CA,17249,"{'type': 'Private', 'carnegieClassification': ...","{'totalGrantAid': 51958388.0, 'percentOfStuden...",Suburban,147100.0
4,Columbia University in the City of New York is...,5,"{'naturalId': 'fred/college/13', 'name': 'Colu...",Columbia University,NY,31456,"{'type': 'Private', 'carnegieClassification': ...","{'totalGrantAid': 42294774.0, 'percentOfStuden...",Urban,132100.0
...,...,...,...,...,...,...,...,...,...,...
595,"Affiliated with the Presbyterian Church, Whitw...",596,"{'naturalId': 'fred/college/208', 'name': 'Whi...",Whitworth University,WA,2814,"{'type': 'Private', 'carnegieClassification': ...","{'totalGrantAid': 22087881.0, 'percentOfStuden...",Suburban,89100.0
596,"Augustana University in Sioux Falls, SD, one o...",597,"{'naturalId': 'fred/college/307', 'name': 'Aug...",Augustana University,SD,2113,"{'type': 'Private', 'carnegieClassification': ...","{'totalGrantAid': 10137731.0, 'percentOfStuden...",Urban,91900.0
597,"A small, Lutheran liberal arts college in the ...",598,"{'naturalId': 'fred/college/121', 'name': 'Lut...",Luther College,IA,1951,"{'type': 'Private', 'carnegieClassification': ...","{'totalGrantAid': 17709147.0, 'percentOfStuden...",Rural,83800.0
598,Coe College is a liberal arts school in Cedar ...,599,"{'naturalId': 'fred/college/224', 'name': 'Coe...",Coe College,IA,1430,"{'type': 'Private', 'carnegieClassification': ...","{'totalGrantAid': 14071402.0, 'percentOfStuden...",Urban,88300.0


#### Adding 'organization' to main table

In [4]:
# add organization
organization = [ x['organization'] for x in responseData]
data_main = pd.concat([data_main, pd.DataFrame(organization)], axis=1)

#### Add single 'Primary_Key' column to main table

In [5]:
# add naturalId column
data_main['Primary_Key'] = [ x['organization']['naturalId'].split('/')[-1] for x in responseData]

#### Adding 'organization/geoLocation' to main table

In [6]:

# add geoLocation columns
rows = []
for organizationsLists in responseData:
    row = {}    
    # row['Primary_Key'] = organizationsLists['organization']['naturalId'].split('/')[-1]
    try:
        row['latutide'] = organizationsLists['organization']['geoLocation']['latitude']
    except:
        pass
    try:
        row['longitude']= organizationsLists['organization']['geoLocation']['longitude']
    except:
        pass
    rows.append(row)
data_main = pd.concat([data_main, pd.DataFrame(rows)], axis=1)

#### Adding 'Academics' to main table

In [7]:
academics = [ # items in the 'academics' key to be unpacked
    'attendanceStatus',
    'firstToSecondYearRetention',
    'overallGraduationRates',
    'enrollmentByGender',
    'graduationRateByGender',
    'enrollmentByRace',
    'graduationRateByRace']

def restructure(list_of_dictionaries,name): # change the shape of the json
    row = {}
    for item in list_of_dictionaries:
        key_value = [value for key,value in item.items()]
        key_name = name + '_' + key_value[0]
        row[key_name] = key_value[1]
    return row

def dataFrame_of_Item(item): # build a data frame from all json's 
    list_of_rows = []
    for organization in responseData:
        # create reshaped json
        graduationRateByRace = restructure(organization['academics'][item],item)
        # add primary_key number
        # graduationRateByRace['PrimaryKey'] = organization['organization']['naturalId'].split('/')[-1]
        # append to list
        list_of_rows.append(graduationRateByRace)
    return pd.DataFrame(list_of_rows)

# add date frames to main data
df_ls = []

for item in academics:
    df_ls.append(dataFrame_of_Item(item))

academics_unpacked = pd.concat(df_ls, axis=1)

data_main = pd.concat([data_main,academics_unpacked], axis=1)



#### Adding 'organization/socialNetworks' to main table

In [8]:
def restructure(list_of_dictionaries): # input: list of dictionaries | output: structured dictionary
    row = {}
    for item in list_of_dictionaries:
        key_value = [value for key,value in item.items()]
        key_name =key_value[0]
        row[key_name] = key_value[1]
    return row

def dataFrame_of_Item(col_name,item,TF): # input: name of dict to unpack | output: data frame
    list_of_rows = []
    for organization in responseData:
        # create json
        dictionary = {}
        try:
            # create reshaped json
            dictionary = dictionary | restructure(organization[col_name][item])
        except:
            pass
        # add primary_key number
        if TF:
            dictionary['Primary_Key'] = organization['organization']['naturalId'].split('/')[-1]
        # append to list
        list_of_rows.append(dictionary)
    return pd.DataFrame(list_of_rows)

df_socialNetworks = dataFrame_of_Item('organization','socialNetworks',True)

data_main = data_main.merge(df_socialNetworks, on='Primary_Key')

#### Adding 'financialAid' to main table

In [9]:
financialAid = [
    'grantAidByType',
    'avgGrantAidByType',
    'loansByType',
    'avgLoansByType']

def restructure(list_of_dictionaries): # input: list of dictionaries | output: structured dictionary
    row = {}
    for item in list_of_dictionaries:
        key_value = [value for key,value in item.items()]
        key_name =key_value[0]
        row[key_name] = key_value[1]
    return row

def dataFrame_of_Item(col_name,item,TF): # input: name of dict to unpack | output: data frame
    list_of_rows = []
    for organization in responseData:
        # create json
        dictionary = {}
        try:
            # create reshaped json
            dictionary = dictionary | restructure(organization[col_name][item])
        except:
            pass
        # add primary_key number
        if TF:
            dictionary['Primary_Key'] = organization['organization']['naturalId'].split('/')[-1]
        # append to list
        list_of_rows.append(dictionary)
    return pd.DataFrame(list_of_rows)

# add date frames to main data
df_ls = []

for item in financialAid:
    df_ls.append(dataFrame_of_Item('financialAid',item,True))

financialAid_unpacked = pd.concat(df_ls, axis=1)

financialAid_unpacked = financialAid_unpacked.loc[:, ~financialAid_unpacked.columns.duplicated()]

data_main = data_main.merge(financialAid_unpacked)

data_main = data_main.loc[:, ~data_main.columns.duplicated()]

drop_columns = ['organization','academics','financialAid','listImages','geoLocation','visible','relatedVisible','imageExists','socialNetworks','collegeMedia']

data_main.drop(drop_columns, axis=1, inplace=True)

In [11]:
data_main.columns


Index(['description', 'rank', 'organizationName', 'state', 'studentPopulation',
       'campusSetting', 'medianBaseSalary', 'naturalId', 'name', 'uri',
       'webSite', 'phoneNumber', 'recentContentCount', 'uris', 'shortUri',
       'squareImage', 'city', 'country', 'region', 'yearFounded', 'stateCode',
       'placeUri', 'landscapeImage', 'industries', 'embargo', 'image',
       'industry', 'ceoName', 'ceoTitle', 'premiumProfile', 'employees',
       'portraitImage', 'Primary_Key', 'latutide', 'longitude',
       'attendanceStatus_partTime', 'attendanceStatus_fullTime',
       'firstToSecondYearRetention_fullTime',
       'firstToSecondYearRetention_partTime', 'overallGraduationRates_4',
       'overallGraduationRates_6', 'enrollmentByGender_enrollmentMale',
       'enrollmentByGender_enrollmentFemale', 'enrollmentByGender_AgeUnder18',
       'enrollmentByGender_Age18to24', 'enrollmentByGender_Age25to64',
       'enrollmentByGender_Age65andOver',
       'graduationRateByGender_gradua

# ------------------ TEST AREA ------------------

# WHAT TO DO:
* ~~unpack 'organization'~~
    * ~~unpack 'geoLocation'~~
    * ~~unpack 'socialNetworks'~~
* ~~unpack 'academics'~~
    * ~~unpack 'attendanceStatus'~~
    * ~~unpack 'firstToSecondYearRetention'~~
    * ~~unpack 'overallGraduationRates'~~
    * ~~unpack 'enrollmentByGender'~~
    * ~~unpack 'graduationRateByGender'~~
    * ~~unpack 'enrollmentByRace'~~
    * ~~unpack 'graduationRateByRace'~~
* ~~unpack 'financialAid'~~
    * ~~grantAidByType~~
    * ~~avgGrantAidByType~~
    * ~~loansByType~~
    * ~~avgLoansByType~~
* ~~drop unnecessary columns~~

In [ ]:
data_main.shape


In [ ]:
col_to_drop = [
       'organization',
       'academics',
       'financialAid',
       'listImages',
       'visible',
       'relatedVisible',
       'imageExists',
       'recentContentCount',
       'country',
       'collegeMedia',
       'landscapeImage',
       'industries',
       'embargo',
       'image',
       'industry',
       'ceoName',
       'ceoTitle',
       'premiumProfile',
       'employees',
       'portraitImage',
       'naturalId',
       'geoLocation',
       'uri',
       'uris',
       'socialNetworks',
       'placeUri'
       ]

data_main.drop(col_to_drop, axis=1, inplace=True)


In [ ]:

data_main.to_csv('University_Data.csv')

In [ ]:
pd.options.display.max_columns = None
data_main

In [ ]:
pd.options.display.max_columns = None
data_main.head()

In [ ]:
data_main[data_main.organizationName == 'Brigham Young University']

In [ ]:
data_main[data_main.organizationName.str.contains('Brigham')]

In [ ]:
pd.options.display.max_rows = 100
# data_main.sort_values(by='enrollmentByGender_enrollmentFemale', ascending=False)

In [ ]:
data_main.columns

In [ ]:
responseData[52]['organization']['socialNetworks']